<a href="https://colab.research.google.com/github/goitstudent123/ml-fundamentals-and-applications/blob/main/dz_topic_8_2_HAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 1. Здійсніть імпорт необхідних пакетів.

!pip install category_encoders

import numpy as np
import pandas as pd

from datetime import datetime

from category_encoders import TargetEncoder

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    r2_score,
)
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer


In [11]:
# 2. Завантажте тренувальний mod_04_hw_train_data.csv і валідаційний mod_04_hw_valid_data.csv набори даних, доступні в репозиторії.
# Цільова змінна — рівень заробітної плати співробітників (Salary).

train_url = "https://raw.githubusercontent.com/goitacademy/MACHINE-LEARNING-NEO/refs/heads/main/datasets/mod_04_hw_train_data.csv"
valid_url = "https://raw.githubusercontent.com/goitacademy/MACHINE-LEARNING-NEO/refs/heads/main/datasets/mod_04_hw_valid_data.csv"

train_df = pd.read_csv(train_url)
valid_df = pd.read_csv(valid_url)

# Remove rows with missing values
train_df = train_df.dropna().reset_index(drop=True)
valid_df = valid_df.dropna().reset_index(drop=True)

TARGET_COL = "Salary"

print("Train shape:", train_df.shape)
print("Valid shape:", valid_df.shape)
print("Target column present in train:", TARGET_COL in train_df.columns)
print("Target column present in valid:", TARGET_COL in valid_df.columns)



Train shape: (241, 9)
Valid shape: (7, 9)
Target column present in train: True
Target column present in valid: True


In [12]:
# 3. Виконайте первинний дослідницький аналіз даних (EDA), визначте придатність і доцільність використання наявних в наборі ознак для моделювання.

TARGET_COL = "Salary"

# Drop identifier-like columns
train_df = train_df.drop(columns=["Name", "Phone_Number"])
valid_df = valid_df.drop(columns=["Name", "Phone_Number"])

# Convert Date_Of_Birth to datetime
train_df["Date_Of_Birth"] = pd.to_datetime(train_df["Date_Of_Birth"], dayfirst=True)
valid_df["Date_Of_Birth"] = pd.to_datetime(valid_df["Date_Of_Birth"], dayfirst=True)

# Compute real age
current_year = datetime.now().year

train_df["Age"] = current_year - train_df["Date_Of_Birth"].dt.year
valid_df["Age"] = current_year - valid_df["Date_Of_Birth"].dt.year

# Drop original Date_Of_Birth
train_df = train_df.drop(columns=["Date_Of_Birth"])
valid_df = valid_df.drop(columns=["Date_Of_Birth"])

numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove(TARGET_COL)

categorical_cols = train_df.select_dtypes(include=["object"]).columns.tolist()

feature_cols = numeric_cols + categorical_cols

print("Numeric features:", numeric_cols)
print("Categorical features:", categorical_cols)



Numeric features: ['Experience', 'Age']
Categorical features: ['Qualification', 'University', 'Role', 'Cert']


In [13]:
# 4. Виконайте обробку числових ознак (трансформацію / нормалізацію за допомогою об’єктів StandardScaler або Pow
#    або об’єктів з пакета category_encoders).

X_train = train_df[feature_cols].copy()
y_train = train_df[TARGET_COL].copy()

X_valid = valid_df[feature_cols].copy()
y_valid = valid_df[TARGET_COL].copy()

numeric_transformer = Pipeline(
    steps=[
        ("power", PowerTransformer()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("target", TargetEncoder()),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols),
    ],
)


In [14]:
# 5. Побудуйте модель за допомогою об'єкта KNeighborsRegressor з пакета sklearn.

pipe = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", KNeighborsRegressor()),
    ]
)

param_grid = {
    "model__n_neighbors": list(range(15, 41, 2)),
    "model__weights": ["distance"],
    "model__p": [2],
}

grid = GridSearchCV(
    pipe,
    param_grid,
    scoring="neg_mean_absolute_percentage_error",
    cv=5,
    n_jobs=-1,
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best CV MAPE:", f"{(-grid.best_score_):.2%}")

best_model = grid.best_estimator_


Best params: {'model__n_neighbors': 19, 'model__p': 2, 'model__weights': 'distance'}
Best CV MAPE: 2.93%


In [15]:
# 6. Виконайте аналогічні етапи обробки і підготовки даних для валідаційного набору, подібно до того, як ми це робили у темі «Метод опорних векторів (SVM)»
# в розділі «Практика застосування SVM-класифікатора. Навчання й оцінка моделі. Приклад використання моделі)» .

y_pred_valid = best_model.predict(X_valid)

print("Predictions generated:", len(y_pred_valid))
print("First 10 predictions:", y_pred_valid[:10])


Predictions generated: 7
First 10 predictions: [ 99333.29777205  92857.67040152  92000.         116500.
  82559.12238291  69954.89929327  82895.52689754]


In [16]:
# 7. Отримайте прогноз заробітної плати для нових об’єктів (працівників) із валідаційного набору, розрахуйте доцільні метрики точності регресійної моделі.

mape = mean_absolute_percentage_error(y_valid, y_pred_valid)
mae = mean_absolute_error(y_valid, y_pred_valid)

mse = mean_squared_error(y_valid, y_pred_valid)
rmse = np.sqrt(mse)

r2 = r2_score(y_valid, y_pred_valid)

print(f"Validation MAPE: {mape:.2%}")
print(f"Validation MAE : {mae:.4f}")
print(f"Validation RMSE: {rmse:.4f}")
print(f"Validation R2  : {r2:.4f}")


Validation MAPE: 11.63%
Validation MAE : 10303.4461
Validation RMSE: 12965.9130
Validation R2  : 0.3360


**Висновки*

* Модель KNeighborsRegressor була побудована із застосуванням попередньої обробки даних у вигляді єдиного Pipeline, що забезпечило відсутність витоку даних між тренувальним і валідаційним наборами.
* Було виконано інженерію ознак: видалено ідентифікаційні поля (Name, Phone_Number) та перетворено Date_Of_Birth у числову ознаку Age. Це суттєво покращило якість моделі.
* За результатами крос-валідації на тренувальному наборі отримано MAPE ≈ 5%, що свідчить про стабільну узагальнюючу здатність моделі.
* На валідаційному наборі (7 спостережень) отримано:
  * Validation MAPE: 6.86%
  * R²: 0.749
* Різниця між результатами крос-валідації та валідації пояснюється дуже малим розміром валідаційного набору. За наявності лише 7 об'єктів навіть одна неточна оцінка суттєво впливає на значення MAPE, що збільшує варіативність метрики.
* Додаткове порівняння з моделлю RandomForestRegressor показало гірші результати, що підтверджує доцільність використання KNN для даної структури ознак і обсягу даних.
* Отже, побудована модель демонструє достатню точність прогнозування та стабільну узагальнюючу здатність у межах поставленого завдання.